# Human Caption

In [ ]:
import pickle

# gt captions
human_captions_path="./Cap3D_human_Objaverse.pkl"

with open(human_captions_path, 'rb') as f:
    human_captions = pickle.load(f)     #dict

In [ ]:
fewshots_uid=[]
for uid in fewshots_uid:
    print(f"{uid}:{human_captions[uid]}\n")

# Qwen3-vl-plus

In [ ]:
import os

def diffurank_select(obj_path):
    diffu_path=os.path.join(obj_path,"diffurank_scores.pkl")

    with open(diffu_path, 'rb') as f:
        diffu_scores = pickle.load(f)     #numpy array

    # 将索引和分数配对
    indexed = list(enumerate(diffu_scores))
    # 按分数排序（稳定排序，分数相同时保持原顺序）
    indexed.sort(key=lambda x: x[1])
    # 取前6个
    lowest_six = indexed[:6]
    # 提取索引
    indices = [idx for idx, val in lowest_six].reverse()

    return indices

In [ ]:
prompt_text="以上是一个3D物体的6张渲染图。"

In [ ]:
import os
import dashscope

# 以下为北京地域base_url，若使用弗吉尼亚地域模型，需要将base_url换成 https://dashscope-us.aliyuncs.com/api/v1
# 若使用新加坡地域的模型，需将base_url替换为：https://dashscope-intl.aliyuncs.com/api/v1
dashscope.base_http_api_url = "https://dashscope.aliyuncs.com/api/v1"


obj_uid=TODO
data_dir="./data"

obj_path=os.path.join(data_dir,obj_uid)

idx_list=diffurank_select(obj_path)

local_path1 = os.path.join(obj_path,f"{idx_list[0]:05}.png")
local_path2 = os.path.join(obj_path,f"{idx_list[1]:05}.png")
local_path3 = os.path.join(obj_path,f"{idx_list[2]:05}.png")
local_path4 = os.path.join(obj_path,f"{idx_list[3]:05}.png")
local_path5 = os.path.join(obj_path,f"{idx_list[4]:05}.png")
local_path6 = os.path.join(obj_path,f"{idx_list[5]:05}.png")

image_path1 = f"file://{local_path1}"
image_path2 = f"file://{local_path2}"
image_path3 = f"file://{local_path3}"
image_path4 = f"file://{local_path4}"
image_path5 = f"file://{local_path5}"
image_path6 = f"file://{local_path6}"

messages = [
    {
        "role": "user",
        "content": [
            {"image": image_path1},
            {"image": image_path2},
            {"image": image_path3},
            {"image": image_path4},
            {"image": image_path5},
            {"image": image_path6},
            {"text": TODO}
        ]
    }
]

response = dashscope.MultiModalConversation.call(
    # 若没有配置环境变量，请用百炼API Key将下行替换为：api_key="sk-xxx"
    # 各地域的API Key不同。获取API Key：https://help.aliyun.com/zh/model-studio/get-api-key
    #api_key=os.getenv('DASHSCOPE_API_KEY'),
    api_key="sk-e7d194c626aa4fcf90aeb0bbdfe59c6b",
    model='qwen3-vl-plus', # 此处以qwen3.5-plus为例，可按需更换模型名称。模型列表：https://help.aliyun.com/zh/model-studio/models
    messages=messages
)

print(response.output.choices[0].message.content[0]["text"])

: 

In [ ]:
"""
You are given 6 renderings of a 3D object, please generate a concise caption that describes it. Captions should typically begin with an article ("a" or "an"), followed by color(s), shape, and the object type.Include distinctive features introduced by "with" when relevant (e.g., parts, textures, accessories). Use simple, everyday vocabulary and mention colors, materials (wooden, metal, plastic, etc.), and any notable details like wheels, windows, eyes, or decorations. Avoid long or complex sentences. The caption should be a short phrase or a simple sentence that captures the essential visual attributes.
"""